## Part a) Box Picking Example in BotOp

In [1]:
import robotic as ry
import numpy as np
import time
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000., 'physx/angularDamping': 10., 'physx/defaultFriction': 100.})
ry.params_print()

physx/motorKp: 10000,
physx/motorKd: 1000,
physx/angularDamping: 10,
physx/defaultFriction: 100

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [3]:
#   .setShape(ry.ST.sphere, size=[.025]) \
C.addFrame('box') \
    .setPosition([-.25,.1,.675]) \
    .setShape(ry.ST.ssBox, size=[.05,.05,.05,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

0

In [4]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

In [5]:
way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0 0 .1) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('d(90 0 0 1)')

C.view()

37

In [6]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [7]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):94.6307 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    71.2095  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    49.3458  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    29.5544  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    12.5532  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    6.60194  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    4.43574  ACCEPT
--newton-- it:   7  |Delta|:   0.193917  alpha:          1  evals:   8  f(y):    3.74957  ACCEPT
--newton-- it:   8  |Delta|:  0.0699611  alpha:          1  evals:   9  f(y):    3.68524  ACCEPT
--newton-- it:   9  |Delta|:  0.0274786  alpha:          1  evals:  10  f(y

In [8]:
komo.view(False, "waypoints solution")

0

In [9]:
komo.view_close()
path = komo.getPath()

In [10]:
bot = ry.BotOp(C, False)
bot.home(C)

In [33]:
bot.gripperMove(ry._left)
#bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [11]:
bot.move(path, [2., 3.])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [12]:
bot.gripperCloseGrasp(ry._left, 'box')
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-box of type rigid with rel [0, 0, 0]


In [13]:
bot.home(C)

In [ ]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [14]:
del bot
del C

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


## Part b) Box Pushing Example

In [15]:
import robotic as ry
import numpy as np
import time

print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

ry version 0.1.10 compile time: Jun 26 2024 18:23:48
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/angularDamping: 10,
physx/defaultFriction: 100,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
KOMO/sparse,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopInners: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
opt/muInit: 1,
opt/muInc: 5,
opt/muMax: 10000,
opt/muLBInit: 0.1,
opt/muLBDec: 0.2,
opt/maxLambda: -1,
opt/constrainedMethod: ,
seed: 0,
bot/useGripper,
bot/blockRealRobot!,
botsim/verbose: 1,
botsim/tau: 0.01,
botsim/hyperSpeed: 1,
botsim/engine: physx,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBody,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
ph

In [16]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [17]:
C.addFrame('box') \
    .setPosition([-.25,.1,0.725]) \
    .setShape(ry.ST.ssBox, size=[.15,.15,.15,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

0

In [18]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(-.07 0 0) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('t(.2 0 0) d(90 0 0 1)')

C.view()

0

In [19]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1])
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

In [24]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=1 ) \
    .solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals:   4  A(x):    3.70964  f:    3.10172  g:          0  h:    2.37816  |x-x'|:   0.283638 	stop:DeltaConverge
==nlp== it:   1  evals:   4  A(x):    7.35716  mu:5
==nlp== it:   1  evals:   8  A(x):    4.61078  f:    4.41697  g:          0  h:   0.303566  |x-x'|:   0.251105 	stop:DeltaConverge
==nlp== it:   2  evals:   8  A(x):    5.03561  mu:25
==nlp== it:   2  evals:  10  A(x):    4.91171  f:    4.57224  g:          0  h:   0.115518  |x-x'|:  0.0230101 	stop:DeltaConverge
==nlp== it:   3  evals:  10  A(x):    6.27176  mu:125
==nlp== it:   3  evals:  12  A(x):    6.27163  f:     4.5727  g:          0  h:   0.113804  |x-x'|: 0.00319516 	stop:DeltaConverge
==nlp== StoppingCriterion Delta<0.01
{ time: 0.004371, evals: 12, done: 1, feasible: 1, sos: 4.5727, f: 0, ineq: 0, eq: 0.113804 }


In [23]:
komo.view(False, "waypoints solution")

0

NO SELECTION: SELECTION DEPTH = 1 200


In [25]:
komo.view_close()
path = komo.getPath()

In [26]:
bot = ry.BotOp(C, False)
bot.home(C)

In [27]:
bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [28]:
bot.moveTo(path[0])
bot.moveTo(path[1])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [29]:
bot.home(C)

In [30]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [31]:
del bot
del C

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
